In [1]:
import os
import cv2
import random
import json
import datetime
import numpy as np
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GroupKFold
import tensorflow as tf
print('tensorflow version:', tf.__version__)

tensorflow version: 2.3.1


In [2]:
## concept of this notebook was from:
# how to deal with DICOM was from
# https://www.raddq.com/dicom-processing-segmentation-visualization-in-python/
# and
# https://www.kaggle.com/vgarshin/osic-keras-images-and-tabular-data-model 

In [3]:
######################################################
## seed and defaults
######################################################

seed = 2020
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
    

DATA_DIR = '../../input/osic-pulmonary-fibrosis-progression'
GROUP_SPLITS = 5

TRAINING_FEATURES = [
    'Female', 
    'Male',
    'Currently smokes', 
    'Ex-smoker', 
    'Never smoked',
    'Percent',
    #'init_week_Percent',
    'Age', 
    'relative_week', 
    'init_week_FVC'
]

SCALED_FEATURES = [
    'Percent', 
    'Age',
    'relative_week', 
    'init_week_FVC'
]

IMG_SIZE = 224
IMG_SLICES = 12
CUTOFF = 2

EPOCHS = 30
BATCH_SIZE = 10
BATCH_PRED = 1
MODEL_NAME = 'dropout_variance'
MODEL_VERSION = 'v11b'
MODEL = MODEL_NAME + '_' + MODEL_VERSION + '_batch_' + str(BATCH_SIZE)

In [4]:
######################################################
## get files and split tabular data
######################################################

train = pd.read_csv(f'{DATA_DIR}/train.csv')
train.drop_duplicates(keep=False, inplace=True, subset=['Patient','Weeks'])

test = pd.read_csv(f'{DATA_DIR}/test.csv')
subm = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

subm['Patient'] = subm['Patient_Week'].apply(lambda x: x.split('_')[0])
subm['Weeks'] = subm['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))

subm =  subm[['Patient','Weeks','Confidence','Patient_Week']]
subm = subm.merge(test.drop('Weeks', axis=1), on='Patient')

train['SPLIT'] = 'train'
test['SPLIT'] = 'test'
subm['SPLIT'] = 'submission'

data = train.append([test, subm])

######################################################
## initial week and relative week augmentations
######################################################

data['init_week'] = data['Weeks']
data.loc[data.SPLIT == 'submission', 'init_week'] = np.nan
data['init_week'] = data.groupby('Patient')['init_week'].transform('min')
data['relative_week'] = data['Weeks'] - data['init_week']

######################################################
## add initial fvc to all patients rows
######################################################

init_fvc = data.groupby('Patient')[['Patient', 'Weeks', 'init_week', 'FVC']].head()
init_fvc = init_fvc.loc[init_fvc.Weeks == init_fvc.init_week]
init_fvc.columns = ['Patient', 'Weeks', 'init_week', 'init_week_FVC']
init_fvc.drop(['Weeks', 'init_week'], axis=1, inplace=True)
data = data.merge(init_fvc, on='Patient', how='left')

del init_fvc


######################################################
## scale the continuous variables
## and dummies of categories
######################################################

min_max_scaler = MinMaxScaler()

data[SCALED_FEATURES] = min_max_scaler.fit_transform(data[SCALED_FEATURES])
data = pd.concat([data, pd.get_dummies(data.Sex), pd.get_dummies(data.SmokingStatus)], axis=1)

######################################################
## add initial percent to all patients rows
######################################################

init_perc = data.groupby('Patient')[['Patient', 'Weeks', 'init_week', 'Percent']].head()
init_perc = init_perc.loc[init_perc.Weeks == init_perc.init_week]
init_perc.columns = ['Patient', 'Weeks', 'init_week', 'init_week_Percent']
init_perc.drop(['Weeks', 'init_week'], axis=1, inplace=True)
data = data.merge(init_perc, on='Patient', how='left')

del init_perc

######################################################
## separate for training, testing, submission
######################################################

train = data.loc[data.SPLIT == 'train']
test = data.loc[data.SPLIT == 'test']
subm = data.loc[data.SPLIT == 'submission']

del data

In [5]:
train.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,SPLIT,Confidence,Patient_Week,init_week,relative_week,init_week_FVC,Female,Male,Currently smokes,Ex-smoker,Never smoked,init_week_Percent
0,ID00007637202177411956430,-4,2315,0.236393,0.769231,Male,Ex-smoker,train,NaN,NaN,-4.0,0.179012,0.241456,0,1,0,1,0,0.236393
1,ID00007637202177411956430,5,2214,0.215941,0.769231,Male,Ex-smoker,train,NaN,NaN,-4.0,0.234568,0.241456,0,1,0,1,0,0.236393
2,ID00007637202177411956430,7,2061,0.184960,0.769231,Male,Ex-smoker,train,NaN,NaN,-4.0,0.246914,0.241456,0,1,0,1,0,0.236393
3,ID00007637202177411956430,9,2144,0.201767,0.769231,Male,Ex-smoker,train,NaN,NaN,-4.0,0.259259,0.241456,0,1,0,1,0,0.236393
4,ID00007637202177411956430,11,2069,0.186580,0.769231,Male,Ex-smoker,train,NaN,NaN,-4.0,0.271605,0.241456,0,1,0,1,0,0.236393


In [6]:

#### image helpers
def get_img_seq(pat_id, slice_count, data_dir, folder, img_size):
        
    images = []

    slices = get_images(pat_id, slice_count, data_dir, folder)
    scans = get_pixels_hu(slices)

    for img_idx in range(slice_count):
        img = scans[img_idx]

        ## resize images to be same shape
        img = cv2.resize(img, (img_size, img_size))

        ## normalize the image pixels
        img = (img - np.min(img)) / (np.max(img) - np.min(img))

        #reshape for tesnor
        img = np.repeat(img[..., np.newaxis], 3, -1)
        images.append(img)     

    return np.array(images).astype(np.float32)
    
def get_pixels_hu(scans):
    '''
    hu pixel is from
    https://www.raddq.com/dicom-processing-segmentation-visualization-in-python/
    '''
    
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)


def get_images(pat_id, slice_count, data_dir, folder):
    c_off = 2
    path = f'{data_dir}/{folder}/{pat_id}'

    file_names = sorted(os.listdir(path), key=lambda x: int(os.path.splitext(x)[0]))

    idxs = [
        int(i * len(file_names) / (slice_count + 2 * c_off)) 
        for i in range(slice_count + 2 * c_off)
    ]

    image_array = [
        pydicom.read_file(path + '/' + file_names[idx])
        for idx in idxs[c_off:-c_off]
    ]

    if len(image_array) < slice_count:
        for i in range(slice_count - len(image_array)):
            image_array.append(pydicom.read_file(path + '/' + os.listdir(path)[-1]))

    return image_array

######################################################
## data generator, used to feed data to TensorFlow in batches
######################################################

class DataGen(tf.keras.utils.Sequence):
    def __init__(
        self, 
        df, 
        tab_features,
        data_dir,
        batch_size=8, 
        mode='fit', 
        shuffle=False, 
        cutoff=2,
        folder='train',
        slice_count=12, 
        img_size=224):

        self.df = df
        self.data_dir = data_dir
        self.shuffle = shuffle
        self.mode = mode
        self.batch_size = batch_size
        self.folder = folder
        self.img_size = img_size
        self.slice_count = slice_count
        self.tab_features = tab_features
        self.on_epoch_end()
        
    def __len__(self):

        return int(np.floor(len(self.df) / self.batch_size))
    
    def on_epoch_end(self):
        
        self.indexes = np.arange(len(self.df))

        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __getitem__(self, index):

        batch_size = min(self.batch_size, len(self.df) - index * self.batch_size)
        
        X_img = np.zeros((batch_size, self.slice_count, self.img_size, self.img_size, 3), dtype=np.float32)
        X_tab = self.df[index * self.batch_size : (index + 1) * self.batch_size][self.tab_features].values
        pats_batch = self.df[index * self.batch_size : (index + 1) * self.batch_size]['Patient'].values
        
        for i, pat_id in enumerate(pats_batch):
            imgs_seq = get_img_seq(pat_id, self.slice_count, self.data_dir, self.folder, self.img_size)
            X_img[i, ] = imgs_seq

        if self.mode == 'fit' or self.mode == 'test':
            y = np.array(
                self.df[index * self.batch_size : (index + 1) * self.batch_size]['FVC'].values, 
                dtype=np.float32
            )

            return (X_img, X_tab), y

        elif self.mode == 'predict':
            y = np.zeros(batch_size, dtype=np.float32)

            return (X_img, X_tab), y


In [7]:
######################################################
## Custom model
## cnn for images and dnn for tabs
######################################################

def get_cnn(inputs_image_shape, dropout_prob, nodes, node_increase, seed):
    print('dropout prob:', dropout_prob, 'nodes:', nodes, 'node increase:', node_increase)
    
    inputs_images = tf.keras.layers.Input(shape=(*inputs_image_shape, ))
    x_images = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(inputs_images)
    x_images = tf.keras.layers.GlobalMaxPooling3D()(x_images)
    x_images = tf.keras.layers.BatchNormalization()(x_images)
    x_images = tf.keras.layers.Dropout(dropout_prob, seed=seed)(x_images)  
    x_images = tf.keras.layers.Dense(int(node_increase * nodes), activation='relu')(x_images)
    x_images = tf.keras.layers.BatchNormalization()(x_images)
    x_images = tf.keras.layers.Dropout(dropout_prob, seed=seed)(x_images)
    x_images = tf.keras.layers.Dense(nodes, activation='relu')(x_images)


    model = tf.keras.Model(inputs_images, x_images)
    
    return model

def get_dnn(inputs_tab_shape, nodes):
    
    inputs_tab = tf.keras.layers.Input(shape=(inputs_tab_shape, ))
    x_tabs = tf.keras.layers.Dense(nodes, activation='relu')(inputs_tab)

    model = tf.keras.Model(inputs_tab, x_tabs)
    
    return model

def get_model(inputs_image_shape, inputs_tab_shape, seed, dropout_prob=.05, nodes=4, node_increase=2, lr=0.1):

    cnn = get_cnn(inputs_image_shape, dropout_prob=dropout_prob, nodes=nodes, node_increase=node_increase, seed=seed)
    dnn = get_dnn(inputs_tab_shape, nodes=nodes)
    
    combinedInput = tf.keras.layers.concatenate([cnn.output, dnn.output])
    
    x = tf.keras.layers.Dense(int(nodes/2), activation="relu")(combinedInput)
    preds = tf.keras.layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inputs=[cnn.input, dnn.input], outputs=preds)
    
    opt = tf.optimizers.Adam(learning_rate=lr)
    model.compile(loss='mean_absolute_error', optimizer=opt)
    
    return model

In [8]:
######################################################
## Since we have groups (patients) this splits
## in a way we don't mix a patient in validation
## and in training at same time
######################################################

group_folds = GroupKFold(n_splits=GROUP_SPLITS)
train['fold'] = -1

for i, (train_idx, val_idx) in enumerate(group_folds.split(train, groups=train['Patient'])):
    train.loc[val_idx, 'fold'] = i

In [9]:
######################################################
## finally the training
######################################################
## model tuning
## hyperparam that can be tuned
dropout = [0.7, 0.5, 0.1]
nodes = 4
node_increase = 2
lr = 0.1

folds_history = []

for i in range(len(dropout)):
    
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5, 
        verbose=0,
        mode='min'
    )

    checkpoing_save = tf.keras.callbacks.ModelCheckpoint(
        f'{MODEL}_fold_{i}.h5', 
        monitor='val_loss', 
        verbose=1, 
        save_best_only=True,
        mode='min'
    )
    
    log_dir = "logs/fit/" + MODEL + '_fold_' + str(i) + '/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


    train_datagen = DataGen(
        df=train.loc[train['fold'] != i], 
        tab_features=TRAINING_FEATURES,
        data_dir=DATA_DIR,
        batch_size=BATCH_SIZE,
        mode='fit', 
        shuffle=True, 
        folder='train',
        slice_count=IMG_SLICES, 
        img_size=IMG_SIZE
    )

    val_datagen = DataGen(
        df=train.loc[train['fold'] == i],
        tab_features=TRAINING_FEATURES,
        data_dir=DATA_DIR,
        batch_size=BATCH_SIZE,
        mode='fit', 
        shuffle=False, 
        folder='train',
        slice_count=IMG_SLICES, 
        img_size=IMG_SIZE
    )
    
    model = get_model(
        inputs_image_shape=(IMG_SLICES, IMG_SIZE, IMG_SIZE, 3), 
        inputs_tab_shape=len(TRAINING_FEATURES),
        seed=seed,
        dropout_prob=dropout[i], 
        nodes=nodes,
        node_increase=node_increase, 
        lr=lr
    )

    history = model.fit(
        train_datagen,
        validation_data=val_datagen,
        batch_size=BATCH_SIZE, 
        epochs=EPOCHS, 
        callbacks=[checkpoing_save, early_stop, tensorboard_callback],
        verbose=1
    )
    
    folds_history.append(history)

dropout prob: 0.7 nodes: 4 node increase: 2
Epoch 1/30
  1/122 [..............................] - ETA: 0s - loss: 2325.1968WARNING:tensorflow:From /home/dev_pool/anaconda3/envs/thesis/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
122/122 [==============================] - ETA: 0s - loss: 1328.6523
Epoch 00001: val_loss improved from inf to 507.76364, saving model to dropout_variance_v11b_batch_10_fold_0.h5
122/122 [==============================] - 64s 525ms/step - loss: 1328.6523 - val_loss: 507.7636
Epoch 2/30
122/122 [==============================] - ETA: 0s - loss: 478.3312
Epoch 00002: val_loss improved from 507.76364 to 391.12793, saving model to dropout_variance_v11b_batch_10_fold_0.h5
122/122 [==============================] - 63s 518ms/step - loss: 478.3312 - val_loss: 391.1279
Epoc

Epoch 10/30
122/122 [==============================] - ETA: 0s - loss: 175.6600
Epoch 00010: val_loss did not improve from 153.99780
122/122 [==============================] - 63s 518ms/step - loss: 175.6600 - val_loss: 155.8015
Epoch 11/30
122/122 [==============================] - ETA: 0s - loss: 160.0197
Epoch 00011: val_loss did not improve from 153.99780
122/122 [==============================] - 63s 517ms/step - loss: 160.0197 - val_loss: 158.1008
Epoch 12/30
122/122 [==============================] - ETA: 0s - loss: 160.9510
Epoch 00012: val_loss did not improve from 153.99780
122/122 [==============================] - 63s 514ms/step - loss: 160.9510 - val_loss: 294.1799
Epoch 13/30
122/122 [==============================] - ETA: 0s - loss: 151.3443
Epoch 00013: val_loss did not improve from 153.99780
122/122 [==============================] - 63s 517ms/step - loss: 151.3443 - val_loss: 168.7215
Epoch 14/30
122/122 [==============================] - ETA: 0s - loss: 169.3071
Epoc

In [11]:
for i in range(len(dropout)):
    h = folds_history[i]
    history_file = f'{MODEL}_fold_{i}_history.txt'
    dict_to_save = {}

    for k, v in h.history.items():
        dict_to_save.update({k: [np.format_float_positional(x) for x in h.history[k]]})

    with open(history_file, 'w') as file:
        json.dump(dict_to_save, file)


In [12]:
model_file = f'{MODEL}_fold_{0}.h5'
model = tf.keras.models.load_model(model_file)

In [13]:
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 12, 224, 224 0                                            
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 12, 224, 224, 12          input_1[0][0]                    
__________________________________________________________________________________________________
global_max_pooling3d (GlobalMax (None, 3)            0           time_distributed[0][0]           
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 3)            12          global_max_pooling3d[0][0]       
_______________________________________________________________________________________

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit